In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import List, TypeVar, Any, Tuple
import pandas as pd
from typing import Generic
from typing import Literal

## First attempt – a dataframe wrapper which holds deltas

In [ ]:
class DataFrameWithDeltas:
    deltas: List[Tuple[str, pd.DataFrame]]

    def __init__(self, initial: pd.DataFrame):
        self.deltas = [("new", initial)]

    def update(self, delta: pd.DataFrame, kind: Literal["new", "extend"]):
        self.deltas.append((kind, delta))

    @property
    def df(self):
        v = None
        for kind, delta in self.deltas:
            if kind == "new":
                v = delta
            elif kind == "extend":
                assert isinstance(v, pd.DataFrame)
                v = pd.concat([v, delta])
            else:
                raise NotImplementedError(f"{kind=}")
        return v


In [ ]:
d = DataFrameWithDeltas(pd.DataFrame({"a": [1,2,3], "b": list("abc")}))
d.df

,a,b
0,1,a
1,2,b
2,3,c


In [ ]:
d.update(pd.DataFrame({"a": [4], "b": list("d")}), kind="extend")
d.df


,a,b
0,1,a
1,2,b
2,3,c
0,4,d


In [ ]:
d.update(pd.DataFrame({"a": [5], "b": list("e")}), kind="new")
d.df

,a,b
0,5,e


In [ ]:
d.deltas

[('new',
     a  b
  0  1  a
  1  2  b
  2  3  c),
 ('extend',
     a  b
  0  4  d),
 ('new',
     a  b
  0  5  e)]

## State with Deltas

literature_search / seed_data -> {conditions: […], observations: […], experimental_data: [...] model: […], status:
“expected”,
variables:
VariableCollection(…), kind = "new", from: "seed"}

experimentalist -> {conditions: […], status: “proposed”, kind="extend", from: "experimentalist"} ## the
experiment_runner only wants the "last" proposed conditions, so we need to be able to access the "last proposed
experiment"

experiment_runner -> {experimental_data: [...], conditions: view(data), observations: view(data), status:
“observed”, kind: "extend", from: "experiment_runner"}

theorist -> {model: TheModel(), (fitted_)data: [...], status: “fitted”, kind: "extend", from: "theorist"}

can "kind" just be the "extension" function rather than the abstract name.

Idea: we store all these results as "deltas":
all_data = [seed_data, experimentalist_result_1, experiment_runner_result_1, theorist_result_1, ... ]
... and then resolve the deltas whenever we need a particular code.


In [ ]:
class State:
    initial: Any
    deltas: List[StateDelta[State]]

S = TypeVar("S")

class StateDelta(S):
    value: S
    kind: Literal["replace", "extend"]


TypeError: TypeVar(name, constraint, ...): constraints must be types. Got (~S,).

In [ ]:
State = TypeVar("S")

class StateWithDeltas(Generic[State]):
    initial: Any
    deltas: List[StateDelta[State]]

class StateDelta(Generic[State]):
    value: State
    kind: Literal[]
